# Data Bootcamp Final Project
####Explaining Music Popularity Using Audio Features and Machine Learning

Done by: Deema Hazim and Ameera Alrahmah

##Which audio features matter most for popularity?

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from datasets import load_dataset

In [ ]:
# Load spotify tracks dataset
dataset = load_dataset("maharshipandya/spotify-tracks-dataset")
df = pd.DataFrame(dataset["train"])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Select the audio feature columns we want to analyze
audio_features = [
    'danceability',
    'energy',
    'loudness',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo'
]

# Select features and target
df_model = df[audio_features + ['popularity']]

In [ ]:
df_model.head()

In [ ]:
df_model.describe()

In [ ]:
# Define features and target
X = df_model[audio_features]
y = df_model['popularity']

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train linear regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

In [ ]:
# Create data frame of feature coefficients (shows the correlation with popularity)
coefficients = pd.DataFrame({
    'Feature': audio_features,
    'Coefficient': lin_reg.coef_
}).sort_values(by='Coefficient', ascending=False)

coefficients

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Generate predictions for training and test sets
y_pred_train = lin_reg.predict(X_train_scaled)
y_pred_test = lin_reg.predict(X_test_scaled)

# Evaluate the model performance on train and test data
# R-squared shows how well the model explains popularity
# while RMSE and MAE indicate the average error in predicted popularity
print("Model Performance:")
print(f"Train R-squared: {r2_score(y_train, y_pred_train):.3f}")
print(f"Test R-squared: {r2_score(y_test, y_pred_test):.3f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")

##**Can we classify songs as “hit” vs “non-hit”?**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Define what makes a song a "hit"
df['is_hit'] = (df['popularity'] >= 50).astype(int)

In [ ]:
print("Hit Distribution:")
print(df['is_hit'].value_counts())

In [ ]:
#Prepare the data (same features as before)
X = df[audio_features]
y = df['is_hit']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
#Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#Logistic Regression
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_scaled, y_train)
y_pred_lr = log_reg.predict(X_test_scaled)

print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.3f}")
print(classification_report(y_test, y_pred_lr))

In [ ]:
#Decision Tree
tree = DecisionTreeClassifier(random_state=42, max_depth=5)
tree.fit(X_train_scaled, y_train)
y_pred_tree = tree.predict(X_test_scaled)

print(f"Accuracy: {accuracy_score(y_test, y_pred_tree):.3f}")
print(classification_report(y_test, y_pred_tree))

In [ ]:
#Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.3f}")
print(classification_report(y_test, y_pred_rf))

In [ ]:
models_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree', 'Random Forest'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_tree),
        accuracy_score(y_test, y_pred_rf)
    ]
})

plt.figure(figsize=(8, 5))
plt.bar(models_comparison['Model'], models_comparison['Accuracy'], color=['blue', 'green', 'orange'])
plt.ylabel('Accuracy')
plt.title('Model Performance Comparison')
plt.ylim([0, 1])
for i, v in enumerate(models_comparison['Accuracy']):
    plt.text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')
plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
#Confusion Matrix for each model
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

#Logistic Regression
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Logistic Regression')
axes[0].set_ylabel('Actual')
axes[0].set_xlabel('Predicted')

#Decision Tree
cm_tree = confusion_matrix(y_test, y_pred_tree)
sns.heatmap(cm_tree, annot=True, fmt='d', cmap='Greens', ax=axes[1])
axes[1].set_title('Decision Tree')
axes[1].set_ylabel('Actual')
axes[1].set_xlabel('Predicted')

#Random Forest
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Oranges', ax=axes[2])
axes[2].set_title('Random Forest')
axes[2].set_ylabel('Actual')
axes[2].set_xlabel('Predicted')

plt.tight_layout()
plt.show()

###Let us see if different hit thresholds matter

In [ ]:
thresholds = [40, 50, 60, 70]

for threshold in thresholds:
    df['is_hit'] = (df['popularity'] >= threshold).astype(int)

    # How many hits?
    hit_pct = df['is_hit'].mean() * 100
    print(f"Threshold {threshold}: {hit_pct:.1f}% of songs are hits")

    # Train model
    y = df['is_hit']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train_scaled, y_train)
    y_pred = rf.predict(X_test_scaled)

    print(f"  Accuracy: {accuracy_score(y_test, y_pred):.3f}\n")


###Let us explore what makes hits different from non-hits

In [ ]:
df['is_hit'] = (df['popularity'] >= 50).astype(int)
hits = df[df['is_hit'] == 1]
non_hits = df[df['is_hit'] == 0]

#Compare average features
comparison = pd.DataFrame({
    'Feature': audio_features,
    'Hits': [hits[f].mean() for f in audio_features],
    'Non-Hits': [non_hits[f].mean() for f in audio_features]
})
comparison['Difference'] = comparison['Hits'] - comparison['Non-Hits']
comparison = comparison.sort_values('Difference', ascending=False)

print(comparison.round(3))

In [ ]:
plt.figure(figsize=(10, 6))
colors = ['green' if x > 0 else 'red' for x in comparison['Difference']]
plt.barh(comparison['Feature'], comparison['Difference'], color=colors)
plt.xlabel('Difference (Hits - Non-Hits)')
plt.title('Hits Have More Green and Less Red')
plt.axvline(x=0, color='black', linestyle='--')
plt.tight_layout()
plt.show()

##**Do genre or artist popularity improve prediction accuracy?**

In [ ]:
#Baseline: Audio Features Only (what we already have)
print("BASELINE: Audio Features Only")

df['is_hit'] = (df['popularity'] >= 50).astype(int)

X_audio = df[audio_features]
y = df['is_hit']

X_train, X_test, y_train, y_test = train_test_split(
    X_audio, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_baseline = RandomForestClassifier(n_estimators=100, random_state=42)
rf_baseline.fit(X_train_scaled, y_train)
y_pred_baseline = rf_baseline.predict(X_test_scaled)

baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
print(f"Accuracy with only audio features: {baseline_accuracy:.3f}")

In [ ]:
#Audio Features + Genre

#Create genre dummies (one-hot encoding)
genre_dummies = pd.get_dummies(df['track_genre'], prefix='genre')

#Combine audio features with genre
X_with_genre = pd.concat([df[audio_features], genre_dummies], axis=1)

X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(
    X_with_genre, y, test_size=0.2, random_state=42
)

scaler_g = StandardScaler()
X_train_g_scaled = scaler_g.fit_transform(X_train_g)
X_test_g_scaled = scaler_g.transform(X_test_g)

rf_genre = RandomForestClassifier(n_estimators=100, random_state=42)
rf_genre.fit(X_train_g_scaled, y_train_g)
y_pred_genre = rf_genre.predict(X_test_g_scaled)

genre_accuracy = accuracy_score(y_test_g, y_pred_genre)
print(f"Accuracy with audio + genre: {genre_accuracy:.3f}")
print(f"Improvement: {(genre_accuracy - baseline_accuracy):.3f} ({((genre_accuracy - baseline_accuracy)/baseline_accuracy)*100:.1f}%)")

In [ ]:
#Audio features + artist popularity

# Calculate artist average popularity
artist_pop = df.groupby('artists')['popularity'].mean().to_dict()
df['artist_avg_popularity'] = df['artists'].map(artist_pop)

# Combine audio features with artist popularity
X_with_artist = df[audio_features + ['artist_avg_popularity']]

X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(
    X_with_artist, y, test_size=0.2, random_state=42
)

scaler_a = StandardScaler()
X_train_a_scaled = scaler_a.fit_transform(X_train_a)
X_test_a_scaled = scaler_a.transform(X_test_a)

rf_artist = RandomForestClassifier(n_estimators=100, random_state=42)
rf_artist.fit(X_train_a_scaled, y_train_a)
y_pred_artist = rf_artist.predict(X_test_a_scaled)

artist_accuracy = accuracy_score(y_test_a, y_pred_artist)
print(f"Accuracy with audio + artist: {artist_accuracy:.3f}")
print(f"Improvement: {(artist_accuracy - baseline_accuracy):.3f} ({((artist_accuracy - baseline_accuracy)/baseline_accuracy)*100:.1f}%)")

In [ ]:
#Audio + Genre + Artist

X_full = pd.concat([df[audio_features], genre_dummies, df[['artist_avg_popularity']]], axis=1)

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_full, y, test_size=0.2, random_state=42
)

scaler_f = StandardScaler()
X_train_f_scaled = scaler_f.fit_transform(X_train_f)
X_test_f_scaled = scaler_f.transform(X_test_f)

rf_full = RandomForestClassifier(n_estimators=100, random_state=42)
rf_full.fit(X_train_f_scaled, y_train_f)
y_pred_full = rf_full.predict(X_test_f_scaled)

full_accuracy = accuracy_score(y_test_f, y_pred_full)
print(f"Accuracy with all features: {full_accuracy:.3f}")
print(f"Improvement: {(full_accuracy - baseline_accuracy):.3f} ({((full_accuracy - baseline_accuracy)/baseline_accuracy)*100:.1f}%)")

In [ ]:
results_df = pd.DataFrame({
    'Model': ['Audio Only', 'Audio + Genre', 'Audio + Artist', 'All Combined'],
    'Accuracy': [baseline_accuracy, genre_accuracy, artist_accuracy, full_accuracy]
})

plt.figure(figsize=(10, 6))
bars = plt.bar(results_df['Model'], results_df['Accuracy'], color=['blue', 'green', 'orange', 'red'])
plt.ylabel('Accuracy')
plt.title('Does Genre or Artist Information Improve Predictions?')
plt.ylim([0.7, max(results_df['Accuracy']) + 0.05])

# Add values on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.3f}',
             ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
#Which Genres Have the Most Hits?

genre_hit_rate = df.groupby('track_genre').agg({
    'is_hit': ['mean', 'count']
}).round(3)
genre_hit_rate.columns = ['Hit_Rate', 'Total_Songs']
genre_hit_rate = genre_hit_rate[genre_hit_rate['Total_Songs'] >= 100]  #cOnly genres with 100+ songs
genre_hit_rate = genre_hit_rate.sort_values('Hit_Rate', ascending=False)

print("\nTop 10 Genres by Hit Rate:")
print(genre_hit_rate.head(10))

In [ ]:
#Visualize top genres
plt.figure(figsize=(12, 6))
top_genres = genre_hit_rate.head(10)
plt.barh(top_genres.index, top_genres['Hit_Rate'], color='purple')
plt.xlabel('Hit Rate')
plt.title('Top 10 Genres Most Likely to Produce Hits')
plt.tight_layout()
plt.show()

##**Do audio features cluster into underlying dimensions that explain why certain features matter for hit prediction?**

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
#Using PCA analysis

# Select the audio features and drop missing values
X = df[audio_features].dropna()

# Standardize features for PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Run PCA
pca = PCA()
pca.fit(X_scaled)

# Display the variance explained by each principal component
variance = pca.explained_variance_ratio_ * 100
cumulative = np.cumsum(variance)

print("Variance explained by each component:")
for i in range(len(variance)):
    print(f"Component {i+1}: {variance[i]:.1f}% (Total: {cumulative[i]:.1f}%)")

In [ ]:
# Plot the variance explained by each principal component
plt.figure(figsize=(10, 5))
plt.bar(range(1, 10), variance, color='skyblue')
plt.xlabel('Component')
plt.ylabel('% Variance Explained')
plt.title('How Much Does Each Component Explain?')
plt.axhline(y=10, color='red', linestyle='--', alpha=0.5)
plt.show()

In [ ]:
# Create table and heatmap of feature loadings for the first three components
# (Only these 3 components are used because they each explain >10% of the variance)
loadings = pd.DataFrame(
    pca.components_[:3].T,
    columns=['Comp 1', 'Comp 2', 'Comp 3'],
    index=audio_features
)

print("\nFeature Loadings:")
print(loadings.round(2))

# Heatmap
plt.figure(figsize=(8, 5))
sns.heatmap(loadings.T, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Which Features Load on Each Component?')
plt.show()

In [ ]:
# Show top 3 features contributing to each of the first three components
print("\nInterpretation:")
print("\nComponent 1 - Top features:")
print(loadings['Comp 1'].abs().sort_values(ascending=False).head(3))

print("\nComponent 2 - Top features:")
print(loadings['Comp 2'].abs().sort_values(ascending=False).head(3))

print("\nComponent 3 - Top features:")
print(loadings['Comp 3'].abs().sort_values(ascending=False).head(3))

In [ ]:
# Reduce to first 3 components
pca_3 = PCA(n_components=3)
components = pca_3.fit_transform(X_scaled)

# Add popularity
df_simple = pd.DataFrame({
    'Comp 1': components[:, 0],
    'Comp 2': components[:, 1],
    'Comp 3': components[:, 2],
    'popularity': df.loc[X.index, 'popularity'].values
})

# Compute correlation of each component with popularity
print("\nCorrelation with Popularity:")
for col in ['Comp 1', 'Comp 2', 'Comp 3']:
    corr = df_simple[col].corr(df_simple['popularity'])
    print(f"{col}: {corr:.3f}")